In [1]:
import os, sys
from pathlib import Path

def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")

src_path = find_src_folder()
sys.path.append(str(src_path))

In [2]:
from agents.fairy.fairy_agent import graph_builder as fairy_graph_builder
from agents.fairy.fairy_interaction_agent import (
    graph_builder as interaction_graph_builder,
)

from langgraph.graph import StateGraph, START, END, MessagesState
from agents.fairy.util import add_human_message
from langchain_core.messages import HumanMessage,AIMessage
from core.game_dto.z_muck_factory import MockFactory


class TempState(MessagesState):
    pass


async def run_fairy_graph(state: TempState):
    graph = fairy_graph_builder.compile()
    mock_player = MockFactory.create_dungeon_player(1)
    config = {"configurable": {"thread_id": "user_1"}}
    result2 = await graph.ainvoke(
        {
            "messages": [
                HumanMessage(content='배고파', additional_kwargs={'created_at': '2025-11-28T22:15:31.616296'}, response_metadata={}, id='11e45572-0cf7-46d1-aaed-a189e91120a5') ,
                AIMessage(content='체력이 많이 낮은 상태에서 던전을 탐험하는 것은 위험해 보여요. 안전한 곳으로 이동하시는 것이 좋을 것 같아요.', additional_kwargs={'created_at': '2025-11-28T22:15:32.689761', 'intent_types': ['SMALLTALK']}, response_metadata={}, id='24172f5e-1a8f-4223-a211-ca1ab0bf0f59'), 
                add_human_message(content="방금 내가 요청한게 뭐였지?")],
            "dungenon_player": mock_player,
        },
        config=config,
    )
    print(result2["messages"])
    return {}


async def run_interaction_graph(state: TempState):
    graph = interaction_graph_builder.compile()
    
    await graph.ainvoke(
        {
            "inventory": [0, 21, 42],
            "messages": [HumanMessage("한손검으로 교체해줘")],
        }
    )
    
    return {}


builder = StateGraph(TempState)
builder.add_node("run_fairy_graph", run_fairy_graph)
builder.add_node("run_interaction_graph", run_interaction_graph)

builder.add_edge(START, "run_fairy_graph")
builder.add_edge(START, "run_interaction_graph")
builder.add_edge("run_fairy_graph", END)
builder.add_edge("run_interaction_graph", END)
graph = builder.compile()

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instanc

In [3]:
result = await graph.ainvoke({"messages":[HumanMessage(content="내가 방금 요청한 말이 뭐였지?")]})


의도 포함 True
체크 메모리 {True}
멀티턴 {True}
[HumanMessage(content='배고파', additional_kwargs={'created_at': '2025-11-28T22:15:31.616296'}, response_metadata={}, id='11e45572-0cf7-46d1-aaed-a189e91120a5'), AIMessage(content='체력이 많이 낮은 상태에서 던전을 탐험하는 것은 위험해 보여요. 안전한 곳으로 이동하시는 것이 좋을 것 같아요.', additional_kwargs={'created_at': '2025-11-28T22:15:32.689761', 'intent_types': ['SMALLTALK']}, response_metadata={}, id='24172f5e-1a8f-4223-a211-ca1ab0bf0f59'), HumanMessage(content='방금 내가 요청한게 뭐였지?', additional_kwargs={'created_at': '2025-11-28T22:35:48.260681'}, response_metadata={}, id='76740ac4-11cf-449e-8b24-fb47497472a0'), AIMessage(content='체력이 많이 낮은 상태에서 던전을 탐험하는 것은 위험해 보여요. 안전한 곳으로 이동하시는 것이 좋을 것 같아요.', additional_kwargs={'created_at': '2025-11-28T22:35:50.058792', 'intent_types': ['SMALLTALK']}, response_metadata={}, id='e92c1add-3f59-4c0c-a8b6-4f960bda30cb')]
